### Libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler

%matplotlib inline

### Utility

In [4]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import confusion_matrix

ftwo_scorer = make_scorer(fbeta_score, beta=2)

def confusion_matrix_scorer(clf, X, y):

     y_pred = clf.predict(X)
     cm = confusion_matrix(y, y_pred)

     return {'tn': cm[0, 0], 'fp': cm[0, 1],
             'fn': cm[1, 0], 'tp': cm[1, 1]}

def false_neg_scorer(clf, X, y):

     y_pred = clf.predict(X)
     cm = confusion_matrix(y, y_pred)
     
     return cm[1, 0]

def false_pos_scorer(clf, X, y):

     y_pred = clf.predict(X)
     cm = confusion_matrix(y, y_pred)
     
     return cm[0, 1]

### Loading and preparing the dataset

In [5]:
DATA_DIRECTORY = ""

In [7]:
cleaned_data_full = pd.read_csv(DATA_DIRECTORY+"cleaned_data_full.csv", skiprows=1)
cleaned_data_full.head()

,Unnamed: 0,Nom,Prénom,Date Naissance,Age,Sexe,IPP,Venue,Entrée venue,Unnamed: 9,...,4,délai.3,LATA,délai.4,mortalité J7,mortalité J30,Mortalité 6 mois,GOSE J30,GOSE 6 mois,Remarques
0,19,GARRIGUES,BERNARD,11/19/1941,NaN,M,338510.0,2.000010e+12,8/5/2020 21:36,NaN,...,0.0,0.0,0.0,0.0,0,0,0,8 Upper Good Recovery (Upper GR),8 Upper Good Recovery (Upper GR),NaN
1,20,KONIECZNY CONTAMIN,LILIAN,11/21/1968,NaN,M,111247.0,2.000010e+12,8/6/2020 11:50,NaN,...,0.0,0.0,0.0,0.0,0,0,0,8 Upper Good Recovery (Upper GR),8 Upper Good Recovery (Upper GR),NaN
2,22,AYEUL,JULIEN,6/14/1997,NaN,M,72080966.0,2.000010e+12,8/7/2020 21:31,NaN,...,0.0,0.0,0.0,0.0,0,0,0,NaN,max,NaN
3,27,MALICHEVA,MARIA,8/5/1978,NaN,F,72081026.0,2.000010e+12,8/8/2020 19:57,NaN,...,0.0,0.0,0.0,0.0,0,nd,nd,nd,nd,NaN
4,29,ELEZAAR,KHALIFA,11/17/1986,NaN,M,72012454.0,2.000010e+12,8/9/2020 2:19,NaN,...,0.0,0.0,0.0,0.0,0,0,0,8 Upper Good Recovery (Upper GR),8 Upper Good Recovery (Upper GR),NaN


In [9]:
# Exclude rows where the 'exclusion' column is not null
cleaned_data_full = cleaned_data_full[cleaned_data_full['Exclusion'].isnull()]

In [11]:
print(cleaned_data_full.columns.tolist())

['Unnamed: 0', 'Nom', 'Prénom', 'Date Naissance', 'Age', 'Sexe', 'IPP', 'Venue', 'Entrée venue', 'Unnamed: 9', 'Exclusion', 'SAMU ', 'GRADE annoncé', 'Mécanisme accident ', 'Pénétrant', "Site d'accueil ", 'PAS  SMUR ', 'PAD  SMUR ', 'FC SMUR ', 'FR SMUR', 'Shock Index SMUR', 'GCS SMUR ', 'GCS (M) SMUR ', 'Hémocue SMUR ', 'Shock Index inversé', 'Shock index diastolique', 'Anomalie pupille SMUR', 'Fracas bassin', 'Amputation', 'ACR SMUR', 'Hémorragie ext SMUR', 'Ischémie', 'Intubation prehosp', 'Expansion volémique', 'OsmoTH prehosp', 'Vasopresseur prehosp', 'PAS DCA', 'PAD DCA', 'FC  DCA', 'Shock index DCA', 'Shock Index inversé.1', 'Shock index diastolique.1', 'GCS DCA', 'GCS (M) DCA', 'Température DCA', 'Hémocue DCA', 'Dextro DCA (mmol/l)', 'DTC Vd', 'DTC IP', 'Osmothérapie', '1 : AIS Tête', '1 : AIS Tête.1', '1 : AIS Tête.2', '1 : AIS Tête.3', 'AIS rachis', 'AIS thorax', 'AIS abdo', 'AIS extrémités', 'ISS', 'IGS ', 'Admission ICU', 'Ventilation', 'ICP', 'Osmotherapy', 'CSF/EVD', 'dee

In [225]:
# Select only some columns 
X_traumatrix = cleaned_data_full_filtered[['PAS  SMUR ', 'GCS SMUR ', 'Hémorragie ext SMUR', 'Vasopresseur prehosp', 
                                           'SDH', 'Petechiae (P)', 'Intrap H (IPH)', 'Subarach (SAH)', '']]
X_traumatrix.head()

,PAS SMUR,PAD SMUR,FC SMUR,FR SMUR,Shock Index SMUR,GCS SMUR,GCS (M) SMUR,Hémocue SMUR,Shock Index inversé,Shock index diastolique,Anomalie pupille SMUR,Fracas bassin,Amputation,ACR SMUR,Hémorragie ext SMUR,Ischémie,Intubation prehosp,Expansion volémique,OsmoTH prehosp,Vasopresseur prehosp
0,190,103,137,nd,0.72,15,6,nd,1.39,1.33,0,0.0,0.0,0.0,0.0,0.0,0.0,500,0.0,0.0
1,87,49,56,nd,0.64,15,6,nd,1.55,1.14,0,0.0,0.0,0.0,0.0,0.0,0.0,250,0.0,0.0
2,100,60,100,17,1,15,6,nd,1,1.67,0,0.0,0.0,0.0,0.0,0.0,0.0,250,0.0,0.0
3,101,64,120,nd,1.19,14,6,13.1,0.84,1.88,0,0.0,0.0,0.0,0.0,0.0,0.0,250,0.0,0.0
4,110,71,107,18,0.97,15,6,15.8,1.03,1.51,0,0.0,0.0,0.0,0.0,0.0,0.0,500,0.0,0.0


In [227]:
# Convert all variables in X_prehosp to numeric, coercing invalid entries to NaN
X_traumatrix_numeric = X_traumatrix.apply(pd.to_numeric, errors="coerce")

# Count missing values (NA) for each variable in X_prehosp
na_counts = X_traumatrix_numeric.isna().sum()

# Get unique values for each variable in X_prehosp to check for potential outliers
unique_values = {col: X_traumatrix_numeric[col].unique() for col in X_traumatrix_numeric.columns}

# Calculate min and max for each variable in X_prehosp_numeric
min_values = X_traumatrix_numeric.min()
max_values = X_traumatrix_numeric.max()

# Create the summary DataFrame with min, max, missing values, and unique values
summary = pd.DataFrame({
    "Variable": X_traumatrix_numeric.columns,
    "Missing Values": na_counts,
    "Unique Values": [list(unique_values[col]) for col in X_traumatrix_numeric.columns],
    "Min Value": min_values,
    "Max Value": max_values
})

# Display the summary
print(summary)

                                        Variable  Missing Values  \
PAS  SMUR                             PAS  SMUR               40   
PAD  SMUR                             PAD  SMUR               50   
FC SMUR                                 FC SMUR               42   
FR SMUR                                  FR SMUR             513   
Shock Index SMUR                Shock Index SMUR              54   
GCS SMUR                               GCS SMUR               16   
GCS (M) SMUR                       GCS (M) SMUR               30   
Hémocue SMUR                       Hémocue SMUR              251   
Shock Index inversé          Shock Index inversé              53   
Shock index diastolique  Shock index diastolique              62   
Anomalie pupille SMUR      Anomalie pupille SMUR              15   
Fracas bassin                      Fracas bassin              13   
Amputation                            Amputation              11   
ACR SMUR                                ACR SMUR

In [231]:
# Définir les limites maximales pour les colonnes
capping_limits = {
    "Shock Index SMUR": 3,
    "GCS SMUR ": 15,
    "GCS (M) SMUR ": 6,
    "Shock Index inversé": 3,
    "Shock index diastolique": 3,
    "Amputation": 1,
    "ACR SMUR": 1,
    "Hémorragie ext SMUR": 1,
    "Ischémie": 1,
    "Intubation prehosp": 1,
    "OsmoTH prehosp": 1,
    "Vasopresseur prehosp": 1
}

# Appliquer le capping
for column, max_value in capping_limits.items():
    if column in X_prehosp_numeric.columns:
        X_prehosp_numeric[column] = X_prehosp_numeric[column].clip(upper=max_value)
    else:
        print(f"Warning: Column '{column}' not found in DataFrame.")

In [233]:
# Définir les stratégies d’imputation
imputation_strategies = {
    "PAS  SMUR ": "median",
    "PAD  SMUR ": "median",
    "FC SMUR ": "median",
    "Shock Index SMUR": "median",
    "GCS SMUR ": "median",
    "GCS (M) SMUR ": "median",
    "Shock Index inversé": "median",
    "Shock index diastolique": "median",
    "Anomalie pupille SMUR": 0,
    "Fracas bassin": 0,
    "Amputation": 0,
    "ACR SMUR": 0,
    "Hémorragie ext SMUR": 0,
    "Ischémie": 0,
    "Intubation prehosp": 0,
    "Expansion volémique": "median",
    "OsmoTH prehosp": 0,
    "Vasopresseur prehosp": 0
}

# Appliquer l’imputation
for column, strategy in imputation_strategies.items():
    if column in X_prehosp_numeric.columns:
        if strategy == "median":
            X_prehosp_numeric[column] = X_prehosp_numeric[column].fillna(X_prehosp_numeric[column].median())
        else:
            X_prehosp_numeric[column] = X_prehosp_numeric[column].fillna(strategy)
    else:
        print(f"Warning: Column '{column}' not found in DataFrame.")

# Vérifiez les résultats de l’imputation
print("Imputation terminée.")


Imputation terminée.


In [235]:
na_counts = X_traumatrix_numeric.isna().sum()
print(na_counts)

PAS  SMUR                    0
PAD  SMUR                    0
FC SMUR                      0
FR SMUR                    513
Shock Index SMUR             0
GCS SMUR                     0
GCS (M) SMUR                 0
Hémocue SMUR               251
Shock Index inversé          0
Shock index diastolique      0
Anomalie pupille SMUR        0
Fracas bassin                0
Amputation                   0
ACR SMUR                     0
Hémorragie ext SMUR          0
Ischémie                     0
Intubation prehosp           0
Expansion volémique          0
OsmoTH prehosp               0
Vasopresseur prehosp         0
dtype: int64


In [237]:
# Colonnes à supprimer
columns_to_drop = ["FR SMUR", "Hémocue SMUR "]

# Suppression si elles existent dans le DataFrame
columns_existing = [col for col in columns_to_drop if col in X_prehosp_numeric.columns]
X_prehosp_numeric = X_prehosp_numeric.drop(columns=columns_existing)

### TILSUM

In [239]:
TIL = pd.read_csv(DATA_DIRECTORY+"cleaned_data_full.csv", usecols=range(71,76))
TIL.head()

,TIL 0,TIL 1,TIL 2,TIL 3,TIL 4
0,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0


In [241]:
# Create y based on the conditions: TIL 2 = 1 or TIL 3 = 1 or TIL 4 = 1
y = pd.DataFrame()
y["y"] = ((TIL.iloc[:, 2] == 1) | (TIL.iloc[:, 3] == 1) | (TIL.iloc[:, 4] == 1)).astype(int)

# Verify the first few rows of y
print(y.head())

# Outcome event
event_count = (y == 1.00).sum()
print(f"outcome events : {event_count}")

   y
0  0
1  0
2  0
3  0
4  0
outcome events : y    46
dtype: int64


In [243]:
# Create y based on the conditions and propagate NA values
y = pd.DataFrame(index=TIL.index)  # Keep the same indexing as TIL

# Apply the conditions, setting NA in y if there are any NA values in the relevant TIL columns
y["y"] = TIL.iloc[:, [0, 1, 2, 3, 4]].apply(
    lambda row: 1 if (row.iloc[2] == 1 or row.iloc[3] == 1 or row.iloc[4] == 1) else 0, axis=1
)

# Set y to NaN if any NA exists in the relevant columns
y.loc[TIL.iloc[:, [0, 1, 2, 3, 4]].isnull().any(axis=1), "y"] = pd.NA

# Verify the first few rows of y
print(y.head())

     y
0  NaN
1  0.0
2  0.0
3  0.0
4  0.0


In [245]:
# Align indexes between y and X_prehosp_numeric
X_prehosp_numeric, y = X_prehosp_numeric.align(y, join="inner", axis=0)

# Identify rows where any NaN exists in y
nan_and_nd_indexes = y.loc[y.isna().any(axis=1)].index

# Drop rows with NaN from both y and X_prehosp_numeric
y = y.drop(index=nan_and_nd_indexes)
X_prehosp_numeric = X_prehosp_numeric.drop(index=nan_and_nd_indexes)

# Check if the number of rows matches
assert X_prehosp_numeric.shape[0] == y.shape[0], "Number of rows in X and y do not match!"

print(f"Number of rows after cleaning: {X_prehosp_numeric.shape[0]}")


Number of rows after cleaning: 545


In [247]:
y = y.to_numpy().ravel()  # Convert y to a 1D array

In [249]:
print(type(y), y.shape)  # Type should be numpy.ndarray and shape should be (n_samples,)

<class 'numpy.ndarray'> (545,)


In [251]:
FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y)

In [ ]:
pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_prehosp_numeric, y, scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, cv=outer_cv, n_jobs=-1)

print("segmentation volumes: HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')


### TIER

In [161]:
TIER = pd.read_csv(DATA_DIRECTORY+"cleaned_data_full.csv", usecols=range(60,70), skiprows=1)
TIER.head()

,Admission ICU,Ventilation,ICP,Osmotherapy,CSF/EVD,deep sedation,Paralysis,Raise CPP/MAP challenge,Barbiturates,CraniectomyHypothermie
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [163]:
# Create y based on the conditions and propagate NA values
y = pd.DataFrame(index=TIER.index)  # Keep the same indexing as TIL

# Apply the conditions, setting NA in y if there are any NA values in the relevant TIL columns
y["y"] = TIER.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].apply(
    lambda row: 1 if (row.iloc[6] == 1 or row.iloc[7] == 1 or row.iloc[8] == 1 or row.iloc[9] == 1) else 0, axis=1
)

# Set y to NaN if any NA exists in the relevant columns
y.loc[TIER.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]].isnull().any(axis=1), "y"] = pd.NA

# Verify the first few rows of y
print(y.head())

     y
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0


In [165]:
# Outcome event
event_count = (y == 1.00).sum()
print(f"outcome events : {event_count}")

outcome events : y    33
dtype: int64


In [207]:
# Align indexes between y and X_prehosp_numeric
X_prehosp_numeric, y = X_prehosp_numeric.align(y, join="inner", axis=0)

# Identify rows where any NaN exists in y
nan_and_nd_indexes = y.loc[y.isna().any(axis=1)].index

# Drop rows with NaN from both y and X_prehosp_numeric
y = y.drop(index=nan_and_nd_indexes)
X_prehosp_numeric = X_prehosp_numeric.drop(index=nan_and_nd_indexes)

# Check if the number of rows matches
assert X_prehosp_numeric.shape[0] == y.shape[0], "Number of rows in X and y do not match!"

print(f"Number of rows after cleaning: {X_prehosp_numeric.shape[0]}")

y = y.to_numpy().ravel()  # Convert y to a 1D array

print(type(y), y.shape)  # Type should be numpy.ndarray and shape should be (n_samples,)

FOLDS = 5
N_REPEATS = 3
nb_total_samples = len(y)

Number of rows after cleaning: 558
<class 'numpy.ndarray'> (558,)


In [209]:
pipeline_smote_under = Pipeline(steps=[('over', SMOTE()), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])
#pipeline_smote_under = Pipeline(steps=[('over', SMOTENC(categorical_features=["fracas_du_bassin", "amputation"])), ('under', RandomUnderSampler(sampling_strategy=0.5)), ('model', HistGradientBoostingClassifier())])


inner_cv = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=5, random_state=1)

p_grid = {"model__learning_rate": [0.01, 0.05, 0.08, 0.1, 0.2, 0.3, 0.5, 1], "over__sampling_strategy": [0.1, 0.2, 0.3], "over__k_neighbors":[3,5,8], "under__sampling_strategy":[0.3, 0.5, 0.7]}
clf = GridSearchCV(estimator=pipeline_smote_under, param_grid=p_grid, scoring={'F2':ftwo_scorer}, refit='F2', cv=inner_cv)

outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

nested_scores_smote_undersampling = cross_validate(clf, X_prehosp_numeric, y, scoring={'F2':ftwo_scorer, 'ROC_AUC':'roc_auc', 'Recall':'recall_macro', 'F1':'f1', 'Brier':"neg_brier_score", 'False_neg_scorer':false_neg_scorer, 'False_pos_scorer':false_pos_scorer}, cv=outer_cv, n_jobs=-1)

print("segmentation volumes: HistGradientBoostingClassifier with hyperparameter gridsearch")

roc_auc_metric = np.mean(nested_scores_smote_undersampling["test_ROC_AUC"])
roc_auc_metric_std = np.std(nested_scores_smote_undersampling["test_ROC_AUC"])
print(f'AUC (max): {np.round(roc_auc_metric, 2)} +- {np.round(roc_auc_metric_std, 2)}')

f1_score = np.mean(nested_scores_smote_undersampling["test_F1"])
f1_score_std = np.std(nested_scores_smote_undersampling["test_F1"])
print(f'F1 Score (max): {np.round(f1_score, 2)} +- {np.round(f1_score_std, 2)}')

f2_score = np.mean(nested_scores_smote_undersampling["test_F2"])
f2_score_std = np.std(nested_scores_smote_undersampling["test_F2"])
print(f'F2 Score (max): {np.round(f2_score, 2)} +- {np.round(f2_score_std, 2)}')

brier_score = -np.mean(nested_scores_smote_undersampling["test_Brier"])
brier_score_std = -np.std(nested_scores_smote_undersampling["test_Brier"])
print(f'Brier Score (min): {np.round(brier_score, 2)} +- {np.round(brier_score_std, 2)}')

# test_False_neg_scorer returns the number of test false negatives -> to get a % we need to divide by the number of test samples*100
false_neg_score = np.mean(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
false_neg_score_std = np.std(nested_scores_smote_undersampling["test_False_neg_scorer"])*100/(nb_total_samples/FOLDS) 
print(f'False negative: {int(np.round(false_neg_score, 0))}% +- {int(np.round(false_neg_score_std, 0))}')

false_pos_score = np.mean(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
false_pos_score_std = np.std(nested_scores_smote_undersampling["test_False_pos_scorer"])*100/(nb_total_samples/FOLDS)
print(f'False positive: {int(np.round(false_pos_score, 0))}% +- {int(np.round(false_pos_score_std, 0))}')

/opt/anaconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


segmentation volumes: HistGradientBoostingClassifier with hyperparameter gridsearch
AUC (max): 0.83 +- 0.09
F1 Score (max): 0.38 +- 0.07
F2 Score (max): 0.54 +- 0.1
Brier Score (min): 0.11 +- -0.01
False negative: 1% +- 1
False positive: 13% +- 4
